In [6]:
import MeCab
from bs4 import BeautifulSoup
import requests
import logging
import jaconv
from gensim import corpora
from gensim import models
from gensim.models import word2vec
from janome.tokenizer import Tokenizer
from pprint import pprint
import pandas as pd, numpy as np

In [9]:
m = MeCab.Tagger('-d //usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd/')
url = "https://www.aozora.gr.jp/access_ranking/2019_xhtml.html"
URL = ""
res = requests.get(url)
res.encoding = 'shift-jis'
soup = BeautifulSoup(res.content, "html.parser")
url_list = [url["href"] for i, url in enumerate(soup.find_all("a", target="_blank")) if i < 50]

In [10]:
title = []
name = []
text = []
for url in url_list:
    res = requests.get(url)
    url_start = url[:37]
    res.encoding = 'shift-jis'
    soup = BeautifulSoup(res.content, "html.parser")
    for i, a in enumerate(soup.find_all("a")):
        if i == 7:
            url_end = a["href"][1:]
    url = url_start + url_end
    res = requests.get(url)
    res.encoding = 'shift-jis'
    soup = BeautifulSoup(res.content, "html.parser")
    title.append(soup.find("h1").string)
    name.append(soup.find("h2").string)
    for tag in soup.find_all(["rt", "rp"]):
        tag.decompose()
    text.append(soup.find("div",{'class': 'main_text'}).get_text())

In [11]:
df = pd.DataFrame({'タイトル': title, '作者': name, '本文': text})

In [12]:
df.head()

,タイトル,作者,本文
0,〔雨ニモマケズ〕,宮澤賢治,\r\n雨ニモマケズ\r\n風ニモマケズ\r\n雪ニモ夏ノ暑サニモマケヌ\r\n丈夫ナカラダ...
1,走れメロス,太宰治,\r\n メロスは激怒した。必ず、かの邪智暴虐の王を除かなければならぬと決意した。メロスには...
2,山月記,中島敦,\r\n 隴西の李徴は博学才穎、天宝の末年、若くして名を虎榜に連ね、ついで江南尉に補せられた...
3,こころ,夏目漱石,\n上 先生と私\n\n\n一\n\r\n 私はその人を常に先生と呼んでいた。だからここでも...
4,羅生門,芥川龍之介,\r\n ある日の暮方の事である。一人の下人が、羅生門の下で雨やみを待っていた。\r\n 広...


In [14]:
text = df['本文'][0]

In [15]:
t = Tokenizer()
ame = jaconv.kata2hira(text)

In [16]:
ame

'\r\n雨にもまけず\r\n風にもまけず\r\n雪にも夏の暑さにもまけぬ\r\n丈夫なからだをもち\r\n慾はなく\r\n決して瞋らず\r\nいつもしづかにわらってゐる\r\n一日に玄米四合と\r\n味噌と少しの野菜をたべ\r\nあらゆることを\r\nじぶんをかんじょうに入れずに\r\nよくみききしわかり\r\nそしてわすれず\r\n野原の松の林のの\r\n小さな萓ぶきの小屋にゐて\r\n東に病気のこどもあれば\r\n行って看病してやり\r\n西につかれた母あれば\r\n行ってその稲の朿を［＃「朿を」はまま］負ひ\r\n南に死にさうな人あれば\r\n行ってこはがらなくてもいゝといひ\r\n北にけんくゎやそしょうがあれば\r\nつまらないからやめろといひ\r\nひどりのときはなみだをながし\r\nさむさのなつはおろおろあるき\r\nみんなにでくのぼーとよばれ\r\nほめられもせず\r\nくにもされず\r\nさういふものに\r\nわたしはなりたい\n\r\n南無無辺行菩薩\r\n南無上行菩薩\r\n南無多宝如来\r\n南無妙法蓮華経\r\n南無釈迦牟尼仏\r\n南無浄行菩薩\r\n南無安立行菩薩\n\n\n\n'

In [17]:
print(m.parse(ame))

	記号,一般,*,*,*,*,*
雨	名詞,一般,*,*,*,*,雨,アメ,アメ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
も	助詞,係助詞,*,*,*,*,も,モ,モ
まけ	動詞,自立,*,*,一段,未然形,まける,マケ,マケ
ず	助動詞,*,*,*,特殊・ヌ,連用ニ接続,ぬ,ズ,ズ
	記号,一般,*,*,*,*,*
風	名詞,一般,*,*,*,*,風,カゼ,カゼ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
も	助詞,係助詞,*,*,*,*,も,モ,モ
まけ	動詞,自立,*,*,一段,未然形,まける,マケ,マケ
ず	助動詞,*,*,*,特殊・ヌ,連用ニ接続,ぬ,ズ,ズ
	記号,一般,*,*,*,*,*
雪	名詞,一般,*,*,*,*,雪,ユキ,ユキ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
も	助詞,係助詞,*,*,*,*,も,モ,モ
夏	名詞,一般,*,*,*,*,夏,ナツ,ナツ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
暑さ	名詞,固有名詞,一般,*,*,*,暑さ,アツサ,アツサ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
も	助詞,係助詞,*,*,*,*,も,モ,モ
まけ	動詞,自立,*,*,一段,未然形,まける,マケ,マケ
ぬ	助動詞,*,*,*,特殊・ヌ,基本形,ぬ,ヌ,ヌ
	記号,一般,*,*,*,*,*
丈夫	名詞,形容動詞語幹,*,*,*,*,丈夫,ジョウブ,ジョーブ
な	助動詞,*,*,*,特殊・ダ,体言接続,だ,ナ,ナ
からだ	名詞,一般,*,*,*,*,からだ,カラダ,カラダ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
もち	名詞,一般,*,*,*,*,もち,モチ,モチ
	記号,一般,*,*,*,*,*
慾	名詞,一般,*,*,*,*,慾,ヨク,ヨク
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
なく	助動詞,*,*,*,特殊・ナイ,連用テ接続,ない,ナク,ナク
	記号,一般,*,*,*,*,*
決して	副詞,一般,*,*,*,*,決して,ケッシテ,ケッシテ
瞋	名詞,一般,*,*,*,*,*
ら	名詞,接尾,一般,*,*,*,ら,ラ,ラ
ず	助動詞,*,*,*,特殊・ヌ,連用ニ接続,ぬ,ズ,ズ
	記号,一般,*,*,*,*,*
いつも	副詞,一般,*,*,*,*,いつも,イツモ,イツモ
しづか	名詞,固有名詞,人